#### Second iteration used clean engineered dataset from 2025 global jobs trends obtained from Kaggle on 10/06/2025. 
-- This is the seoncd iteration for Descriptive stats and EDA for the dataa being used to model precedent based causality for salary and category of jobs globally, and by country 
-- "How does salary change for an increase in avaailable jobs by category globally (by country). 

### import libraries, load data, data review
### What is the data?

In [ ]:
# Load dataset with CSV first, then Excel fallback
import os
import pandas as pd

csv_path = 'df_clean2.csv'
#excel_path = 'dfall_clean.xlsx'

def load_data():
    if os.path.exists(csv_path):
        print(f"Found {csv_path}, loading CSV...")
        df_clean2 = pd.read_csv(csv_path)
    #elif os.path.exists(excel_path):
        #print(f"CSV not found. Found {excel_path}, loading Excel...")
        #df = pd.read_excel(excel_path)
    #else:
        #raise FileNotFoundError(f"Neither {csv_path} nor {excel_path} were found in the current directory.")
    print('Dataframe shape:', df_clean2.shape)
    display(df_clean2.head())
    return df_clean2

# Execute load
df = load_data()

In [ ]:
# Imports
import os, re, datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

sns.set_theme()

In [ ]:
# Quick verification of loaded dataframe
print('DataFrame info:')
print('-'*40)
df.info()
print('\nFirst 5 rows:')
print('-'*40)
display(df.head())

In [ ]:
# Load data
file_name = "global_job_trend_clean.csv"
if not os.path.exists(file_name):
    print(f"File not found: {file_name}. Current working directory: {os.getcwd()}")
else:
    df = pd.read_csv(file_name)
    print(f"Loaded: {file_name}")
    print("Shape:", df.shape)
    print("Head:")
    print(df.head())

    # Understand data
    print("\nDataFrame info:")
    df.info()
    print("\nDimensions (ndim):", df.ndim)
    print("Shape:", df.shape)
    print("Column names:", df.columns.tolist())
    print("Data types:\n", df.dtypes)

## Make intial data tyes updates Dates to date_time

In [ ]:
# Convert columns containing '_date_' to datetime where appropriate (skip date-part integers)
date_like_cols = [c for c in df.columns if '_date_' in c]

part_suffixes = ('_year', '_month', '_day', '_iso_year', '_iso_week')
converted, skipped = [], []

for col in date_like_cols:
    cname = col.lower()
    if cname.endswith(part_suffixes):
        skipped.append(col)  # keep date-part integers as-is
        continue
    if pd.api.types.is_datetime64_any_dtype(df[col]):
        skipped.append(col)  # already datetime
        continue
    if pd.api.types.is_numeric_dtype(df[col]):
        skipped.append(col)  # avoid converting numeric parts to epoch dates
        continue
    df[col] = pd.to_datetime(df[col], errors='coerce')
    converted.append(col)

print(f"Converted to datetime: {converted if converted else 'None'}")
print(f"Skipped (parts/already datetime/numeric): {skipped if skipped else 'None'}")

In [ ]:
df.head()

In [ ]:
# Convert all object dtype columns in df to category
obj_cols = df.select_dtypes(include="object").columns.tolist()

if not obj_cols:
    print("No object columns to convert.")
else:
    df[obj_cols] = df[obj_cols].apply(lambda s: s.astype("category"))
    print(f"Converted object -> category for {len(obj_cols)} columns:")
    print(obj_cols)

In [ ]:
df.info()

## Observe missing data

In [ ]:
# Step 1: Check for missing values in all columns
missing_summary = df.isna().sum().sort_values(ascending=False)
print("Missing values in key columns:")
print(missing_summary)

## Start Decsriptive statistics, summary and visual explanations

In [ ]:
df.describe()

### Summarize missing values

In [ ]:
# Step 1: Check for missing values in key columns
missing_summary = df.isnull().sum()
print("Missing values in columns:")
print(missing_summary)

In [ ]:
# Visualize missing data: counts per column and missingness heatmap

# Use existing summary if available, otherwise compute
_missing_counts = missing_summary.sort_values(ascending=False) if 'missing_summary' in globals() else df.isna().sum().sort_values(ascending=False)
_missing_pct = (_missing_counts / len(df) * 100).round(2)

fig, axes = plt.subplots(1, 2, figsize=(16, 6), constrained_layout=True)

# Barplot of missing counts (with % annotations)
sns.barplot(x=_missing_counts.values, y=_missing_counts.index, ax=axes[0], palette='viridis')
axes[0].set_title('Missing values by column')
axes[0].set_xlabel('Missing count')
axes[0].set_ylabel('')
for i, (count, pct) in enumerate(zip(_missing_counts.values, _missing_pct.values)):
    axes[0].text(count, i, f'  {count} ({pct}%)', va='center')

# Heatmap of missingness pattern (first N rows for readability)
_n = min(600, len(df))
sns.heatmap(df.head(_n).isna(), ax=axes[1], cbar=False, yticklabels=False, cmap='viridis')
axes[1].set_title(f'Missingness pattern (first {_n} rows)')
axes[1].set_xlabel('Columns')
axes[1].set_ylabel('Rows')

plt.show()

In [ ]:
df.info()

### Look at the 5 momnets for undestanding data quality

In [ ]:
# Distributions: histograms for numeric, bar charts for categorical
import math
import matplotlib as mpl

# Reduce font sizes and increase padding to avoid overlapping text in subplots
mpl.rcParams.update({
    "axes.titlesize": 10,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "figure.titlesize": 12,
    "axes.titlepad": 6,
    # Use constrained layout with extra breathing room
    "figure.constrained_layout.use": True,
    "figure.constrained_layout.w_pad": 6.0 / 72.0,
    "figure.constrained_layout.h_pad": 6.0 / 72.0,
    "figure.constrained_layout.wspace": 0.10,
    "figure.constrained_layout.hspace": 0.10,
})
import seaborn as sns
import matplotlib.pyplot as plt

# 1) Numeric variable histograms
num_cols = df.select_dtypes(include=["number"]).columns.tolist()
if num_cols:
    n = len(num_cols)
    cols = 3 if n >= 3 else n
    rows = math.ceil(n / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5.2, rows * 3.6), constrained_layout=True)
    axes = axes.flatten() if n > 1 else [axes]

    for i, col in enumerate(num_cols):
        sns.histplot(df[col].dropna(), bins=30, kde=True, ax=axes[i], color="steelblue")
        axes[i].set_title(str(col))
        axes[i].set_xlabel("")
        axes[i].set_ylabel("Count")

    # Hide any unused axes
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    fig.suptitle("Numeric distributions (hist + KDE)", fontsize=14)
    fig.savefig("histograms_numeric.png", dpi=220, bbox_inches="tight")
    plt.show()
    print(f"Saved numeric histograms to histograms_numeric.png ({n} features).")
else:
    print("No numeric columns found for histograms.")

# 2) Categorical variable bar charts (top 20 levels per column)
cat_cols = df.select_dtypes(include=["category", "object", "bool"]).columns.tolist()
if cat_cols:
    n = len(cat_cols)
    cols = 3 if n >= 3 else n
    rows = math.ceil(n / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 6.0, rows * 4.2), constrained_layout=True)
    axes = axes.flatten() if n > 1 else [axes]

    for i, col in enumerate(cat_cols):
        vc = df[col].astype(str).value_counts(dropna=False).head(20)
        sns.barplot(x=vc.values, y=vc.index, ax=axes[i], palette="viridis")
        axes[i].set_title(f"{col} (top 20)")
        axes[i].set_xlabel("Count")
        axes[i].set_ylabel("")

    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    fig.suptitle("Categorical distributions (top 20 levels)", fontsize=14)
    fig.savefig("barplots_categorical.png", dpi=220, bbox_inches="tight")
    plt.show()
    print(f"Saved categorical bar plots to barplots_categorical.png ({n} features).")
else:
    print("No categorical/object/bool columns found for bar plots.")

### Data Spread

In [ ]:
# Data spread exploration: summary stats, boxplots for numeric columns, and salary spread by key categories

# 1) Summary of spread for numeric columns (use existing num_cols if available)
_s_num_cols = [c for c in num_cols if c != 'id'] if 'num_cols' in globals() else df.select_dtypes(include=['number']).columns.tolist()
if not _s_num_cols:
    print("No numeric columns found for spread analysis.")
else:
    _rows = []
    for _c in _s_num_cols:
        _s = pd.to_numeric(df[_c], errors='coerce').dropna()
        if _s.empty:
            continue
        _q1 = _s.quantile(0.25)
        _q2 = _s.quantile(0.50)
        _q3 = _s.quantile(0.75)
        _iqr = _q3 - _q1
        _mad = float(np.median(np.abs(_s - _q2)))
        _cv = float(_s.std(ddof=1) / _s.mean()) if _s.mean() != 0 else np.nan
        _lo = _q1 - 1.5 * _iqr
        _hi = _q3 + 1.5 * _iqr
        _out = int(((_s < _lo) | (_s > _hi)).sum())
        _rows.append({
            "column": _c,
            "count": int(_s.size),
            "mean": float(_s.mean()),
            "std": float(_s.std(ddof=1)),
            "var": float(_s.var(ddof=1)),
            "min": float(_s.min()),
            "q1": float(_q1),
            "median": float(_q2),
            "q3": float(_q3),
            "iqr": float(_iqr),
            "max": float(_s.max()),
            "mad_median": _mad,
            "cv": _cv,
            "outliers_iqr_1_5x": _out,
        })
    spread_summary = pd.DataFrame(_rows).set_index("column").sort_index()
    print("Numeric spread summary:")
    display(spread_summary)

    # 2) Boxplots for numeric columns (except id)
    _plot_cols = _s_num_cols
    _ncols = 3 if len(_plot_cols) >= 3 else len(_plot_cols)
    _nrows = (len(_plot_cols) + _ncols - 1) // _ncols if _ncols else 0
    if _nrows > 0:
        fig_bp, axs_bp = plt.subplots(_nrows, _ncols, figsize=(_ncols * 5.4, _nrows * 3.8), constrained_layout=True)
        axs_bp = axs_bp.flatten() if isinstance(axs_bp, np.ndarray) else [axs_bp]
        for _i, _c in enumerate(_plot_cols):
            sns.boxplot(x=df[_c], ax=axs_bp[_i], color="steelblue", whis=1.5, fliersize=2)
            axs_bp[_i].set_title(f"{_c} — boxplot")
            axs_bp[_i].set_xlabel("")
            axs_bp[_i].set_xlabel(_c)
        for _j in range(_i + 1, len(axs_bp)):
            axs_bp[_j].axis("off")
        fig_bp.suptitle("Spread of numeric features — boxplots", fontsize=13)
        fig_bp.savefig("spread_boxplots_numeric.png", dpi=220, bbox_inches="tight")
        plt.show()
        print("Saved spread_boxplots_numeric.png")

# 3) Salary spread by key categories (top levels)
if 'avg_salary' in df.columns:
    _group_candidates = []
    if 'cat_cols' in globals():
        _group_candidates = [c for c in ['country', 'category_combined_clean'] if c in cat_cols]
    else:
        for _cand in ['country', 'category_combined_clean']:
            if _cand in df.columns:
                _group_candidates.append(_cand)

    _top_k = 12
    for _gcol in _group_candidates:
        _mask = df['avg_salary'].notna() & df[_gcol].notna()
        if not _mask.any():
            continue
        _order = df.loc[_mask, _gcol].value_counts().head(_top_k).index.tolist()
        if not _order:
            continue
        _sub = df.loc[_mask & df[_gcol].isin(_order), ['avg_salary', _gcol]].copy()

        plt.figure(figsize=(9, 0.5 * len(_order) + 2))
        sns.boxplot(
            data=_sub,
            x='avg_salary',
            y=_gcol,
            order=_order,
            showfliers=False,
            palette='viridis'
        )
        plt.title(f"avg_salary spread by {_gcol} (top {_top_k})")
        plt.xlabel("avg_salary")
        plt.ylabel(_gcol)
        _out_file = f"spread_avg_salary_by_{_gcol}.png"
        plt.savefig(_out_file, dpi=220, bbox_inches="tight")
        plt.show()
        print(f"Saved {_out_file}")

        # Optional: per-group IQR table
        def _iqr_fn(s):
            return s.quantile(0.75) - s.quantile(0.25)
        _grp_stats = _sub.groupby(_gcol)['avg_salary'].agg(
            count='size', mean='mean', std='std', q1=lambda s: s.quantile(0.25),
            median='median', q3=lambda s: s.quantile(0.75), iqr=_iqr_fn
        ).loc[_order]
        print(f"avg_salary spread by '{_gcol}' (top {_top_k}) — summary:")
        display(_grp_stats.round(2))
else:
    print("'avg_salary' not found; skipping salary-by-category spread.")

### Kurtosis and extreme values

In [ ]:
# Skewness and kurtosis for avg_salary only
# Override numeric columns list to just ['avg_salary']
num_cols = ['avg_salary'] if 'avg_salary' in df.columns else []

# Use existing numeric columns list if available; otherwise detect
_num_cols = num_cols if 'num_cols' in globals() else df.select_dtypes(include=['number']).columns.tolist()
if not _num_cols:
    print("No numeric columns found for skewness/kurtosis.")
else:
    X = df[_num_cols].apply(pd.to_numeric, errors='coerce')

    skew = X.skew()
    kurt_excess = X.kurt()           # Fisher definition: Normal => 0
    counts = X.count()

    skew_kurt_summary = (
        pd.DataFrame({
            "count": counts,
            "skewness": skew,
            "excess_kurtosis": kurt_excess,
            "pearson_kurtosis": kurt_excess + 3,  # Normal => 3
        })
        .assign(abs_skew=lambda d: d["skewness"].abs(),
                abs_excess_kurtosis=lambda d: d["excess_kurtosis"].abs())
        .sort_values("abs_skew", ascending=False)
    )

    print("Skewness and kurtosis summary (sorted by |skew|):")
    display(skew_kurt_summary)

    # Bar plots for top features by |skew| and |excess kurtosis|
    top_k = 12
    top_sk = skew_kurt_summary.head(top_k).iloc[::-1]  # reverse for nicer barh order
    top_kt = (
        skew_kurt_summary.sort_values("abs_excess_kurtosis", ascending=False)
        .head(top_k)
        .iloc[::-1]
    )

    fig_sk, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, max(5, 0.5 * top_k)), constrained_layout=True)

    sns.barplot(x=top_sk["skewness"], y=top_sk.index, ax=ax1, palette="coolwarm")
    ax1.axvline(0, color="k", lw=1)
    ax1.set_title("Skewness (top by |skew|)")
    ax1.set_xlabel("Skewness")
    ax1.set_ylabel("")

    sns.barplot(x=top_kt["excess_kurtosis"], y=top_kt.index, ax=ax2, palette="magma")
    ax2.axvline(0, color="k", lw=1)
    ax2.set_title("Excess kurtosis (top by |excess|)")
    ax2.set_xlabel("Excess kurtosis")
    ax2.set_ylabel("")

    out_png = "skew_kurtosis_bars.png"
    fig_sk.savefig(out_png, dpi=220, bbox_inches="tight")
    plt.show()
    print(f"Saved {out_png}")

    # Optional: save table
    out_csv = "skew_kurtosis_summary.csv"
    skew_kurt_summary.to_csv(out_csv, index=True)
    print(f"Saved {out_csv}")

### Correlation Matrix

In [ ]:
# Correlation matrix heatmap for numeric features
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Select numeric columns only
num_cols = df.select_dtypes(include=["number"]).columns.tolist()
if len(num_cols) < 2:
    print("Not enough numeric columns to compute a correlation matrix.")
else:
    corr = df[num_cols].corr()

    # Dynamic figure size based on number of columns
    n = len(num_cols)
    size = max(8, min(0.55 * n + 4, 20))

    # Mask the upper triangle to reduce clutter
    mask = np.triu(np.ones_like(corr, dtype=bool))

    plt.figure(figsize=(size, size))
    sns.heatmap(
        corr,
        mask=mask,
        cmap="RdBu_r",
        center=0,
        annot=(n <= 15),   # annotate only if manageable number of vars
        fmt=".2f",
        square=True,
        linewidths=0.5,
        cbar_kws={"shrink": 0.8, "label": "Pearson r"}
    )
    plt.title("Correlation matrix (numeric features)")
    plt.xticks(rotation=45, ha="right")
    plt.yticks(rotation=0)
    # Using constrained layout via rcParams; avoid tight_layout to prevent layout-engine conflict with colorbar.

    # Save and display
    plt.savefig("corr_matrix.png", dpi=220, bbox_inches="tight")
    plt.show()

    print(f"Saved heatmap to corr_matrix.png with {n} numeric features.")

### Scatter plots

In [ ]:
# Pairwise x-y scatter matrices with regression lines (no duplicates)
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Select numeric columns
num_cols = df.select_dtypes(include=["number"]).columns.tolist()

if len(num_cols) < 2:
    print("Not enough numeric columns for a pairwise scatter matrix (need >= 2).")
else:
    # Limit number of variables for readability/performance (choose by variance)
    max_vars = 10  # adjust if you want more/less
    if len(num_cols) > max_vars:
        var_order = df[num_cols].var(numeric_only=True).sort_values(ascending=False)
        selected_cols = var_order.index[:max_vars].tolist()
        print(f"Selected top {max_vars} numeric columns by variance for plotting:\n{selected_cols}")
    else:
        selected_cols = num_cols

    # Optional row sampling for very large datasets
    max_rows = 5000
    if len(df) > max_rows:
        dplot = df[selected_cols].sample(max_rows, random_state=42)
        print(f"Sampled {max_rows} rows from {len(df)} for speed.")
    else:
        dplot = df[selected_cols]

    # Build pairplot with regression in lower triangle only (no duplicates)
    sns.set_theme()
    g = sns.pairplot(
        dplot,
        kind="reg",
        diag_kind="kde",
        corner=True,              # show only lower triangle (no duplicates)
        plot_kws={
            "scatter_kws": {"alpha": 0.35, "s": 16, "edgecolor": "none"},
            "line_kws": {"color": "crimson", "lw": 1.5},
            "ci": 95,
        },
        height=2.0,               # size of each subplot
        aspect=1.0,
    )

    # Tweak labels/spacing and save
    g.fig.suptitle(
        f"Pairwise scatter with regression (lower triangle) — {len(selected_cols)} vars",
        y=1.02,
        fontsize=13,
    )
    g.fig.tight_layout()
    g.fig.savefig("scatter_matrix_reg.png", dpi=220, bbox_inches="tight")
    plt.show()
    print("Saved scatterplot matrix to scatter_matrix_reg.png.")

### categorical count and propotion

In [ ]:
# Categorical counts and proportions + bar/pie visualization
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Config
viz_type = "bar"   # choose: "bar" or "pie"
top_n = 10          # show top N categories per column
include_other = True

# Identify categorical-like columns
cat_cols = df.select_dtypes(include=["category", "object", "bool"]).columns.tolist()
cat_cols = [c for c in cat_cols if df[c].notna().sum() > 0]

if not cat_cols:
    print("No categorical/object/bool columns found.")
else:
    # Build tidy summary table: column, category, count, proportion_pct
    def summarize_categoricals(frame, columns, top=10, add_other=True):
        parts = []
        for col in columns:
            s = frame[col].astype("string").fillna("<NA>")
            vc = s.value_counts(dropna=False)
            total = int(vc.sum())
            take = vc.head(top)
            if add_other and len(vc) > top:
                other_count = int(total - int(take.sum()))
                take.loc["Other"] = other_count
            part = pd.DataFrame({
                "column": col,
                "category": take.index,
                "count": take.values,
            })
            part["proportion_pct"] = (part["count"] / total * 100.0).round(2)
            parts.append(part)
        return pd.concat(parts, ignore_index=True)

    cat_summary = summarize_categoricals(df, cat_cols, top=top_n, add_other=include_other)
    display(cat_summary.head(30))

    # Plot grid
    n = len(cat_cols)
    cols = 3 if n >= 3 else n
    rows = math.ceil(n / cols)
    figsize = (cols * 6.2, rows * (4.2 if viz_type == "bar" else 4.8))
    fig, axes = plt.subplots(rows, cols, figsize=figsize, constrained_layout=True)
    axes = axes.flatten() if n > 1 else [axes]

    for i, col in enumerate(cat_cols):
        data = cat_summary[cat_summary["column"] == col].copy()
        data = data.sort_values("count", ascending=False)

        if viz_type == "bar":
            sns.barplot(x="count", y="category", data=data, ax=axes[i], palette="viridis")
            axes[i].set_title(f"{col} — top {min(top_n, len(data))}")
            axes[i].set_xlabel("Count")
            axes[i].set_ylabel("")
            # annotate with percentage
            for p, pct in zip(axes[i].patches, data["proportion_pct"].values):
                width = p.get_width()
                y = p.get_y() + p.get_height() / 2
                axes[i].text(width, y, f"  {int(width)} ({pct:.1f}%)", va="center")
        else:  # pie
            counts = data["count"].values
            labels = data["category"].astype(str).values
            total = counts.sum()
            def autopct_fmt(p):
                count = int(round(p / 100.0 * total))
                return f"{p:.1f}%\n({count})"
            axes[i].pie(
                counts,
                labels=labels,
                autopct=autopct_fmt,
                startangle=140,
                textprops={"fontsize": 8},
                colors=sns.color_palette("viridis", n_colors=len(counts)),
            )
            axes[i].axis("equal")
            axes[i].set_title(f"{col} — top {min(top_n, len(data))}")

    # Hide unused axes
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    fig.suptitle(
        f"Categorical distributions (top {top_n}) — {viz_type.upper()}\nCounts annotated with %",
        fontsize=14,
    )

    out_name = f"categorical_counts_proportions_{viz_type}.png"
    fig.savefig(out_name, dpi=220, bbox_inches="tight")
    plt.show()
    print(f"Saved figure to {out_name} for {n} categorical columns.")

In [ ]:
df.info()

In [ ]:
df.head()

### Go to file for global job trends .ipynb data descriptive stats